In [ ]:

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv("/content/heart_disease_uci.csv")
df.dropna(inplace=True)


le = LabelEncoder()
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
for i in categorical_cols:
    df[i] = le.fit_transform(df[i])


x = df.drop(columns=["id", "num",'dataset'])

y = (df["num"] > 0).astype(int)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# XGB
a = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
a.fit(x_train, y_train)
b=a.predict(x_test)
acc_xgb = accuracy_score(y_test,b)

print(f"Accuracy: {acc_xgb * 100:.2f}%")


Accuracy: 88.33%


In [4]:
import pickle
with open('model.pkl','wb') as file:
    pickle.dump(a, file)

In [5]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
%%writefile heart_disease.py
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import pickle
with open("model.pkl","rb")as file:
  a=pickle.load(file)
import streamlit as st
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

st.set_page_config(page_title="Heart Disease Risk Predictor", layout="centered")
st.title("❤️ Heart Disease Risk Predictor")

# Load and train model
@st.cache_data
def load_model_and_columns():
    df = pd.read_csv("/content/heart_disease_uci.csv")
    df.dropna(inplace=True)

    le = LabelEncoder()
    for col in ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']:
        df[col] = le.fit_transform(df[col])

    x = df.drop(columns=["id", "num", "dataset"])
    y = (df["num"] > 0).astype(int)

    model = XGBClassifier(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(x, y)
    return model, x.columns.tolist()

model, expected_columns = load_model_and_columns()

# Manual mappings
sex_map = {"Male": 1, "Female": 0}
cp_map = {
    "Typical Angina": 0,
    "Atypical Angina": 1,
    "Non-anginal Pain": 2,
    "Asymptomatic": 3
}
fbs_map = {"Yes (>120 mg/dl)": 1, "No (≤120 mg/dl)": 0}
restecg_map = {
    "Normal": 0,
    "ST-T Wave Abnormality": 1,
    "Left Ventricular Hypertrophy": 2
}
exang_map = {"Yes": 1, "No": 0}
slope_map = {
    "Upsloping": 0,
    "Flat": 1,
    "Downsloping": 2
}
thal_map = {
    "Normal": 1,
    "Fixed Defect": 2,
    "Reversible Defect": 3
}

st.subheader("📝 Enter Your Health Details")

with st.form(key="predict_form"):
    age = st.slider("Age (years)", 20, 80, 50)
    sex = st.selectbox("Sex", list(sex_map.keys()))
    cp = st.selectbox("Chest Pain Type", list(cp_map.keys()))
    trestbps = st.number_input("Resting Blood Pressure (mm Hg)", 80, 200, 120)
    chol = st.number_input("Serum Cholesterol (mg/dl)", 100, 600, 200)
    fbs = st.selectbox("Fasting Blood Sugar", list(fbs_map.keys()))
    restecg = st.selectbox("Resting ECG Results", list(restecg_map.keys()))
    thalach = st.number_input("Maximum Heart Rate Achieved", 60, 220, 150)
    exang = st.selectbox("Exercise Induced Angina", list(exang_map.keys()))
    oldpeak = st.number_input("ST Depression Induced by Exercise", 0.0, 6.0, 1.0)
    slope = st.selectbox("Slope of Peak Exercise ST Segment", list(slope_map.keys()))
    ca = st.slider("Number of Major Vessels (0–4) Colored by Fluoroscopy", 0, 4, 0)
    thal = st.selectbox("Thalassemia Type", list(thal_map.keys()))

    submitted = st.form_submit_button("🔍 Predict")

if submitted:
    input_dict = {
        "age": age,
        "sex": sex_map[sex],
        "cp": cp_map[cp],
        "trestbps": trestbps,
        "chol": chol,
        "fbs": fbs_map[fbs],
        "restecg": restecg_map[restecg],
        "thalach": thalach,
        "exang": exang_map[exang],
        "oldpeak": oldpeak,
        "slope": slope_map[slope],
        "ca": ca,
        "thal": thal_map[thal]
    }

    input_df = pd.DataFrame([input_dict])
    input_df = input_df.reindex(columns=expected_columns)

    prediction = model.predict(input_df)[0]
    confidence = model.predict_proba(input_df)[0][prediction]

    st.markdown("---")
    st.subheader("🧠 Prediction Result")
    if prediction == 1:
        st.error(f"⚠️ Likely to have heart disease.\nConfidence: {confidence * 100:.2f}%")
    else:
        st.success(f"✅ Unlikely to have heart disease.\nConfidence: {confidence * 100:.2f}%")

    st.markdown("You can change the inputs above and click Predict again to try another case.")





Overwriting heart_disease.py


In [ ]:
!streamlit run heart_disease.py > /content/logs.txt 2>&1 &
!curl ipv4.icanhazip.com


SyntaxError: invalid syntax (462014582.py, line 1)

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://twelve-lemons-notice.loca.lt
^C
